# Generate a Boat Design Space

---

Generate a "reasonable" boat design space by exploring a broad swath of the boat parameter space. Identify a "reasonable" region where the AVS is within a decent range.


In [1]:
import grama as gr
import pandas as pd
import numpy as np
import plotly.express as px
import time
import warnings

from boats import *

DF = gr.Intention()

filename_samples = "data/stability-data.csv"


## Setup

---

Define a model to sample a wide range of possible boat parameter space:


In [2]:
md_performance = (
    gr.Model("Boat Performance Characteristics")
    >> gr.cp_function(
        fun=fun_performance,
        var=["H", "W", "n", "d", "f_com"],
        out=[
            "mass",
            "BM",
            "GM",
            "angle",
            "dMdtheta_0",
            "dMdtheta_avs", 
            "M_max", 
            "M_min", 
            "int_M_stable",
        ]
    )
    >> gr.cp_marginals(
        H=dict(dist="uniform", loc=1, scale=2),
        W=dict(dist="uniform", loc=1, scale=3),
        n=dict(dist="uniform", loc=0.5, scale=1.0),
        d=dict(dist="uniform", loc=0.25, scale=0.50),
        f_com=dict(dist="uniform", loc=0.10, scale=0.90),
    )
    >> gr.cp_copula_independence()
)
md_performance

model: Boat Performance Characteristics

  inputs:
    var_det:

    var_rand:
      H: (+0) uniform, {'mean': '2.000e+00', 's.d.': '5.800e-01', 'COV': 0.29, 'skew.': 0.0, 'kurt.': 1.8}
      W: (+0) uniform, {'mean': '2.500e+00', 's.d.': '8.700e-01', 'COV': 0.35, 'skew.': 0.0, 'kurt.': 1.8}
      n: (+0) uniform, {'mean': '1.000e+00', 's.d.': '2.900e-01', 'COV': 0.29, 'skew.': 0.0, 'kurt.': 1.8}
      d: (+0) uniform, {'mean': '5.000e-01', 's.d.': '1.400e-01', 'COV': 0.29, 'skew.': 0.0, 'kurt.': 1.8}
      f_com: (+0) uniform, {'mean': '5.500e-01', 's.d.': '2.600e-01', 'COV': 0.47, 'skew.': 0.0, 'kurt.': 1.8}

    copula:
      Independence copula

  functions:
      f0: ['H', 'W', 'n', 'd', 'f_com'] -> ['mass', 'BM', 'GM', 'angle', 'dMdtheta_0', 'dMdtheta_avs', 'M_max', 'M_min', 'int_M_stable']

### FOR DEBUGGING: Do a single test-run before the full sample


In [3]:
(
    md_performance
    >> gr.ev_df(df=gr.df_make(W=1, H=1, n=0.75, d=0.5, f_com=0.50))
    >> gr.tf_mutate(
        dGZdtheta_0=-DF.dMdtheta_0 / DF.mass / 386
    )
    >> gr.tf_select("GM", "dGZdtheta_0", gr.everything())
)


,GM,dGZdtheta_0,W,H,n,d,f_com,angle,dMdtheta_avs,dMdtheta_0,M_min,BM,int_M_stable,M_max,mass
0,0.113341,0.136932,1,1,0.75,0.5,0.5,NaN,NaN,-0.392809,-0.610513,0.141413,-1.125098,1.213267e-16,0.007432


### Generate the DOE

Generate a space-filling design with support points.


In [4]:
df_sp = (
    md_performance
    >> gr.ev_sample(n=5e3, df_det="nom", seed=101, skip=True)
    >> gr.tf_sp(n=1000, seed=101)
)
df_sp


eval_sample() is rounding n...
Design runtime estimates unavailable; model has no timing data.
tran_sp has selected var = ['f_com', 'n', 'H', 'W', 'd']
tran_sp finished in 125 iterations with distance criterion 9.957e-04


,f_com,n,H,W,d
0,0.465583,0.970254,1.983238,1.341281,0.345922
1,0.600144,1.257356,2.458964,3.685142,0.302721
2,0.798888,0.994748,1.351817,2.756009,0.520595
3,0.730135,0.884476,1.037399,2.041664,0.759664
4,0.335102,0.870253,2.858828,1.530926,0.345110
...,...,...,...,...,...
995,0.611745,0.567101,3.009206,2.453964,0.378566
996,0.334447,1.388833,1.282592,2.034175,0.292084
997,0.379852,0.624966,1.886479,3.774344,0.450476
998,0.430976,0.768239,2.030728,3.229470,0.578372


## Execute

---

Execute the model at the support points.


In [5]:
# Try to load the data first
try:
    df_results = pd.read_csv(filename_samples)
    print("Data loaded from file.")
    
# If unavailable, generate the data
except FileNotFoundError:
    # Suppress warnings to clean output
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        
        t0 = time.time()
        df_results = (
            md_performance
            >> gr.ev_df(df=df_sp)
        )
        t1 = time.time()
    
    # Write results to disk
    df_results.to_csv(filename_samples, index=False)
    print("Execution time: {0:4.3f} min".format((t1 - t0) / 60))
df_results


Data loaded from file.


,d,n,f_com,H,W,M_max,mass,dMdtheta_avs,dMdtheta_0,M_min,BM,angle,int_M_stable,GM
0,0.414694,0.969522,0.384112,1.931260,1.390651,5.004726e-16,0.019592,NaN,-1.459991,-4.952630e+00,0.108234,NaN,-8.019885,0.193949
1,0.242507,1.195436,0.656740,2.713299,3.580954,7.277457e+00,0.046730,NaN,9.916728,-5.402567e+00,0.425216,NaN,3.675911,-0.557773
2,0.441223,0.936345,0.672138,1.446188,3.744617,6.550768e+00,0.040910,8.237037,-11.668754,-5.879390e+00,1.073773,1.617128,-5.681558,0.768485
3,0.689483,0.882652,0.648739,1.210221,2.530339,1.149756e+00,0.034526,2.380926,-8.409095,-2.228434e+00,0.764303,1.509121,-2.070254,0.652657
4,0.497305,0.780804,0.235978,2.502149,1.505777,7.886234e-16,0.029039,NaN,-8.644337,-1.401972e+01,0.103794,NaN,-25.533024,0.796536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.277591,0.562650,0.893968,2.662832,3.030807,5.471222e+00,0.028587,NaN,9.738265,-6.066334e-16,0.233400,NaN,11.236121,-0.907202
996,0.406322,1.403215,0.243989,1.820171,1.337757,6.824626e-16,0.021281,NaN,-2.759760,-6.680056e+00,0.113371,NaN,-11.439189,0.366400
997,0.607333,0.636330,0.390791,1.504778,3.372128,1.289096e-15,0.044168,NaN,-24.322574,-9.742923e+00,1.100546,NaN,-17.504023,1.399712
998,0.622833,0.751532,0.386794,2.306757,2.632218,1.910895e-15,0.059512,NaN,-19.836982,-1.574321e+01,0.424443,NaN,-34.614643,0.877600
